<a href="https://colab.research.google.com/github/SravanPuvvada/algae_sensor_cnn/blob/main/algae_sensor_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/SravanPuvvada/algae_sensor_cnn.git

Cloning into 'algae_sensor_cnn'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 5 (delta 0), reused 5 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (5/5), done.


In [1]:
import os
os.getcwd()


'/content'

In [2]:
import sys
sys.path.append("/content/algae_sensor_cnn")


In [3]:
import os
os.listdir("/content")


['.config', 'algae.mat', 'algae_sensor_cnn', 'sample_data']

In [4]:
import os

base = "/content/algae_sensor_cnn"

folders = [
    "src",
    "data/raw",
    "data/processed",
    "notebooks",
    "results"
]

for f in folders:
    os.makedirs(os.path.join(base, f), exist_ok=True)


In [5]:
os.listdir("/content/algae_sensor_cnn")


['results',
 '.gitattributes',
 'notebooks',
 '.git',
 'README.md',
 '.gitignore',
 'data',
 'src']

In [6]:
open("/content/algae_sensor_cnn/src/__init__.py", "w").close()


In [7]:
import sys
sys.path.append("/content/algae_sensor_cnn")


In [8]:
%%writefile /content/algae_sensor_cnn/src/data_loader.py
# ===============================
# data_loader.py
# ===============================
"""
Load and parse MATLAB (.mat) sensor data into clean pandas DataFrames.
Handles nested MATLAB structs safely.
"""

import scipy.io as sio
import numpy as np
import pandas as pd


def load_algae_mat(filepath: str) -> dict:
    """
    Load algae .mat file and return raw MATLAB struct.
    """
    data = sio.loadmat(filepath)
    if 'algae' not in data:
        raise KeyError("Expected 'algae' variable not found in .mat file")
    return data['algae']


def mat_struct_to_dataframe(algae_struct) -> pd.DataFrame:
    """
    Convert nested MATLAB algae struct into a clean pandas DataFrame.
    Each column represents a sensor signal.
    """
    fields = algae_struct.dtype.names
    df_dict = {}

    for field in fields:
        if field == 'comments':
            continue

        entry = algae_struct[field][0, 0]
        if 'data' not in entry.dtype.names:
            continue

        values = np.array(entry['data'][0, 0]).squeeze()

        if values.ndim != 1:
            continue

        df_dict[field] = values

    if not df_dict:
        raise ValueError("No valid numeric time-series found in MATLAB struct")

    min_len = min(len(v) for v in df_dict.values())
    for k in df_dict:
        df_dict[k] = df_dict[k][:min_len]

    return pd.DataFrame(df_dict)

Overwriting /content/algae_sensor_cnn/src/data_loader.py


In [9]:
%%writefile /content/algae_sensor_cnn/src/preprocessing.py
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler

def scale_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """
    Robust scaling with NaN-safe preprocessing.
    """

    # 1. Force numeric
    df = df.apply(pd.to_numeric, errors="coerce")

    # 2. Interpolate time-series gaps
    df = df.interpolate(method="linear", limit_direction="both")

    # 3. Fill remaining NaNs (sensor dropouts)
    df = df.fillna(df.median())

    # 4. Robust scaling (immune to outliers)
    scaler = RobustScaler(quantile_range=(5, 95))
    scaled = scaler.fit_transform(df.values)

    # 5. Clip extreme tails
    scaled = np.clip(scaled, -5, 5)

    return pd.DataFrame(scaled, columns=df.columns)


Overwriting /content/algae_sensor_cnn/src/preprocessing.py


In [10]:
%%writefile /content/algae_sensor_cnn/src/windowing.py
# ===============================
# windowing.py
# ===============================
"""
Sliding window generation for CNN/LSTM models.
"""

import numpy as np
import pandas as pd


def create_sliding_windows(
    df: pd.DataFrame,
    window_size: int = 50,
    stride: int = 1,
    target_col: str = "oxygen"
):
    """
    Create sliding windows for CNN input and corresponding binary labels.

    Label rule:
    A window is labeled 1 if the mean oxygen value inside the window
    is greater than the global oxygen mean, else 0.
    """

    X = []
    y = []

    # Compute global mean once (IMPORTANT)
    global_oxygen_mean = df[target_col].mean()

    # Convert dataframe to numpy for faster slicing
    data_values = df.values
    oxygen_index = df.columns.get_loc(target_col)

    for start in range(0, len(df) - window_size + 1, stride):
        end = start + window_size

        # Input window (all features)
        window = data_values[start:end, :]
        X.append(window)

        # Label from oxygen column (Option B: window average)
        window_oxygen_mean = np.mean(data_values[start:end, oxygen_index])
        label = int(window_oxygen_mean > global_oxygen_mean)
        y.append(label)

    # Convert to numpy arrays
    X = np.array(X)
    y = np.array(y)

    return X, y



Overwriting /content/algae_sensor_cnn/src/windowing.py


In [11]:
%%writefile /content/algae_sensor_cnn/src/windowing_multi.py


import numpy as np
import pandas as pd


def create_sliding_windows_multi(
    df: pd.DataFrame,
    window_size: int = 50,
    stride: int = 1,
    target_cols=("oxygen", "PAM0", "PAM8", "PAM16", "density"),
    min_votes: int = 2
):
    X, y = [], []

    # ---- Ensure numeric ----
    df = df.apply(pd.to_numeric, errors="coerce")

    # ---- Global means (exclude density trend) ----
    global_means = {}
    for col in target_cols:
        if col != "density":
            global_means[col] = df[col].mean()

    # ---- Column indices ----
    values = df.values
    col_idx = {col: df.columns.get_loc(col) for col in target_cols}

    for start in range(0, len(df) - window_size + 1, stride):
        end = start + window_size
        window = values[start:end, :]

        # 🚨 HARD FILTER — skip poisoned windows
        if np.isnan(window).any() or np.isinf(window).any():
            continue

        # ---------- X ----------
        X.append(window)

        # ---------- Oxygen vote ----------
        oxygen_mean = window[:, col_idx["oxygen"]].mean()
        oxygen_vote = int(oxygen_mean > global_means["oxygen"])

        # ---------- Photosynthesis vote ----------
        pam_votes = 0
        for pam in ("PAM0", "PAM8", "PAM16"):
            pam_mean = window[:, col_idx[pam]].mean()
            if pam_mean > global_means[pam]:
                pam_votes += 1

        photo_vote = int(pam_votes >= 2)

        # ---------- Density trend vote ----------
        density_start = window[0, col_idx["density"]]
        density_end = window[-1, col_idx["density"]]
        density_vote = int(density_end > density_start)

        # ---------- Final label ----------
        health_score = oxygen_vote + photo_vote + density_vote
        y.append(int(health_score >= min_votes))

    # ---- Convert safely ----
    X = np.array(X, dtype=np.float32)
    y = np.array(y, dtype=np.float32).reshape(-1, 1)

    return X, y



Overwriting /content/algae_sensor_cnn/src/windowing_multi.py


In [12]:
%%writefile /content/algae_sensor_cnn/src/models.py
# ===============================
# models.py
# ===============================
"""
CNN model definitions for time-series classification.
"""

import tensorflow as tf
from tensorflow.keras import layers, models

def build_cnn_model(input_shape: tuple) -> tf.keras.Model:
  model = models.Sequential([
      layers.Input(shape=(30, 16)),

      layers.Conv1D(32, kernel_size=3, activation="relu"),
      layers.MaxPooling1D(2),

      layers.Conv1D(64, kernel_size=3, activation="relu"),
      layers.MaxPooling1D(2),

      layers.Flatten(),
      layers.Dense(64, activation="relu"),
      layers.Dropout(0.3),

      layers.Dense(1, activation="sigmoid")
  ])

  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
      loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
      metrics=[
          tf.keras.metrics.AUC(curve="PR", name="pr_auc"),
          tf.keras.metrics.AUC(curve="ROC", name="roc_auc"),
          tf.keras.metrics.Recall(name="recall_unhealthy")
      ]
  )
#   call_backs = [
#     tf.keras.callbacks.EarlyStopping(
#         monitor="val_pr_auc",
#         mode="max",
#         patience=8,
#         restore_best_weights=True
#     )
# ]

  print(model.summary())

  return model


Overwriting /content/algae_sensor_cnn/src/models.py


In [13]:
%%writefile /content/algae_sensor_cnn/src/train.py
from sklearn.model_selection import train_test_split
import numpy as np


def train_model(X, y, model, epochs=20, batch_size=2):
  X_train, X_val, y_train, y_val = train_test_split(
      X, y,
      test_size=0.2,
      stratify=y,
      random_state=42
  )
  history = model.fit(
          X_train, y_train,
          validation_data=(X_val, y_val),
          epochs=epochs,
          batch_size=batch_size,
      )

  print("Train:", np.unique(y_train, return_counts=True))
  print("Val:", np.unique(y_val, return_counts=True))
  y_prob = model.predict(X_val).ravel()
  y_true = y_val.ravel()
  print("latest",y_prob , y_true)

  return history ,X_val , y_val


Overwriting /content/algae_sensor_cnn/src/train.py


In [14]:
%%writefile /content/algae_sensor_cnn/src/evaluate.py
# ======================================
# evaluate.py
# ======================================
"""
Model evaluation utilities for imbalanced health detection.

Design principles:
- No threshold optimization on validation data
- Prefer threshold-independent metrics
- Focus on unhealthy (positive) class detection
"""

import numpy as np
from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
)


# --------------------------------------------------
# Threshold-independent evaluation
# --------------------------------------------------
def evaluate_probabilities(y_true, y_prob):
    """
    Computes threshold-independent metrics.

    Returns:
    - ROC-AUC
    - PR-AUC (PRIMARY METRIC)
    """
    y_true = y_true.ravel()
    y_prob = y_prob.ravel()

    return {
        "roc_auc": roc_auc_score(y_true, y_prob),
        "pr_auc": average_precision_score(y_true, y_prob),
    }


# --------------------------------------------------
# Fixed-threshold evaluation (NO optimization)
# --------------------------------------------------
def confusion_at_fixed_threshold(y_true, y_prob, threshold=0.5):
    """
    Evaluate model at a fixed, pre-declared threshold.

    This avoids overfitting caused by threshold tuning.
    """
    y_true = y_true.ravel()
    y_pred = (y_prob >= threshold).astype(int)

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    return {
        "threshold": threshold,
        "TP": int(tp),
        "FP": int(fp),
        "FN": int(fn),
        "TN": int(tn),
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "recall": recall_score(y_true, y_pred, zero_division=0),
        "f1": f1_score(y_true, y_pred, zero_division=0),
    }


# --------------------------------------------------
# Risk-based evaluation (Top-K)
# --------------------------------------------------
def recall_at_top_k(y_true, y_prob, fraction=0.2):
    """
    Recall when flagging the top-K highest risk windows.

    Example:
    fraction=0.2 → "If we investigate top 20% riskiest windows,
    how many unhealthy cases do we catch?"
    """
    y_true = y_true.ravel()
    y_prob = y_prob.ravel()

    k = max(1, int(len(y_prob) * fraction))
    top_idx = np.argsort(y_prob)[-k:]

    recall = y_true[top_idx].sum() / max(1, y_true.sum())

    return {
        "fraction_flagged": fraction,
        "recall_at_k": recall,
        "windows_flagged": k,
    }





Overwriting /content/algae_sensor_cnn/src/evaluate.py


In [15]:
import pandas as pd
from src.data_loader import load_algae_mat, mat_struct_to_dataframe
print("Imports work ✅")

from src.preprocessing import scale_dataframe
print("2nd import works ✅")

from src.windowing import create_sliding_windows
print("3rd import works ✅")


from src.models import build_cnn_model
print("4th import works ✅")

from src.train import train_model
print("5th import works ✅")



Imports work ✅
2nd import works ✅
3rd import works ✅
4th import works ✅
5th import works ✅


In [17]:
# %%writefile /content/algae-sensor-cnn/src/data_loader.py
# %%writefile /content/algae-sensor-cnn/src/preprocessing.py
# %%writefile /content/algae-sensor-cnn/src/windowing.py
# %%writefile /content/algae-sensor-cnn/src/models.py
# %%writefile /content/algae-sensor-cnn/src/train.py
import numpy as np
from src.data_loader import load_algae_mat, mat_struct_to_dataframe
from src.preprocessing import scale_dataframe
from src.windowing_multi import create_sliding_windows_multi
from src.models import build_cnn_model
from src.train import train_model


algae = load_algae_mat("algae.mat")
df = mat_struct_to_dataframe(algae)
df_scaled = scale_dataframe(df)

X,y = create_sliding_windows_multi(df_scaled, window_size=30)
print("X shape:", X.shape)
print("y shape:", y.shape)
print("Unique labels:", np.unique(y))

# Example label (binary health proxy)
#y = (df["oxygen"].values[50:] > df["oxygen"].mean()).astype(int)

model = build_cnn_model(input_shape=X.shape[1:])
history , X_val , y_val = train_model(X, y, model)

##########Evaluation###########

from src.evaluate import (
    evaluate_probabilities,
    confusion_at_fixed_threshold,
    recall_at_top_k,
)

# Predict probabilities
y_val_prob = model.predict(X_val).ravel()

# 1️⃣ Threshold-independent metrics
metrics = evaluate_probabilities(y_val, y_val_prob)
print("Threshold-independent metrics:")
print(metrics)

# 2️⃣ Fixed threshold evaluation (NO tuning)
cm_fixed = confusion_at_fixed_threshold(
    y_val,
    y_val_prob,
    threshold=0.5
)
print("\nFixed-threshold (0.5) metrics:")
print(cm_fixed)

# 3️⃣ Risk-based evaluation
recall_k = recall_at_top_k(
    y_val,
    y_val_prob,
    fraction=0.5
)
print("\nRecall@Top-K:")
print(recall_k)




X shape: (50, 30, 16)
y shape: (50, 1)
Unique labels: [0. 1.]


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 28, 32)         │         1,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 14, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 12, 64)         │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 6, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 384)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │        24,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,481 (126.88 KB)

 Trainable params: 32,481 (126.88 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 0.7407 - pr_auc: 0.6287 - recall_unhealthy: 0.0653 - roc_auc: 0.4531 - val_loss: 0.6784 - val_pr_auc: 1.0000 - val_recall_unhealthy: 0.1429 - val_roc_auc: 1.0000
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.6451 - pr_auc: 0.9116 - recall_unhealthy: 0.5858 - roc_auc: 0.8982 - val_loss: 0.6309 - val_pr_auc: 1.0000 - val_recall_unhealthy: 0.7143 - val_roc_auc: 1.0000
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6237 - pr_auc: 0.9804 - recall_unhealthy: 0.7974 - roc_auc: 0.9050 - val_loss: 0.5858 - val_pr_auc: 1.0000 - val_recall_unhealthy: 0.7143 - val_roc_auc: 1.0000
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.5807 - pr_auc: 0.9856 - recall_unhealthy: 0.9475 - roc_auc: 0.9619 - val_loss: 0.5416 - val_pr_auc: 1.0000 - val_recall_unhealthy: 1.0000 - val_roc_auc: 1.0000
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5423 - pr_auc: 0.9985 - recall_unhealthy: 0.9891 - roc

“Because the dataset is small and constructed using overlapping sliding windows, we avoid optimizing decision thresholds on validation data. Instead, we report threshold-independent metrics (PR-AUC, ROC-AUC) and fixed operating-point recall, along with risk-based Recall@K, which better reflects real-world monitoring use cases.”


To avoid optimistic bias due to small sample size and overlapping time windows, we report threshold-independent metrics (PR-AUC, ROC-AUC) and fixed operating-point recall, rather than optimizing decision thresholds on validation data.




# **Read ME**



🌱 Algae Health Monitoring using CNNs on Multivariate Sensor Data
📌 Project Overview

This project implements an end-to-end machine learning pipeline to monitor algae system health using multivariate time-series sensor data. The system identifies unhealthy operating conditions early and prioritizes high-risk periods for intervention.

# **🎯 Business Problem**

Industrial algae cultivation systems generate continuous sensor data (oxygen, PAM fluorescence, density). Manual monitoring is inefficient, and failures are often detected too late.

Goal:
Automatically detect unhealthy system behavior while minimizing missed failures.

# **🧠 Solution Approach**

Converted raw sensor streams into sliding time windows

Labeled windows using domain-informed voting logic

Trained a 1D CNN to learn temporal health patterns

Evaluated using imbalance-aware, deployment-safe metrics

# 🔍 Key Features

Sliding-window time-series modeling

Robust preprocessing for noisy sensor data

CNN-based temporal feature learning

Risk-based evaluation (Recall@Top-K)

Overfitting-safe validation strategy

# 📊 Evaluation Metrics

PR-AUC (Primary): Measures unhealthy detection quality under imbalance

ROC-AUC (Secondary): Overall separability

Recall@Top-K: Operational usefulness when resources are limited

High Recall@Top-K means most unhealthy states are detected by inspecting only a small fraction of time windows.

🛠 Tech Stack

Python

NumPy, Pandas

Scikit-learn

TensorFlow / Keras

CNNs for time-series

**🚀 Use Cases**

Algae cultivation monitoring

Industrial sensor health detection

Early warning systems

Predictive maintenance pipelines

# **Challenges faced (and how they were handled)**
**⚠ Challenge 1: Severe class imbalance**

Healthy dominates unhealthy

Accuracy and ROC-AUC misleading

✅ Solution:

PR-AUC

Recall-focused evaluation

Risk-based metrics

**⚠ Challenge 2: Sliding-window leakage**

Overlapping windows inflate metrics

✅ Solution:

No threshold tuning

Conservative evaluation

Emphasis on ranking quality, not absolute scores

**⚠ Challenge 3: Sensor noise & NaNs**

Density and PAM signals noisy

Scaling caused NaNs initially

✅ Solution:

RobustScaler

Explicit NaN checks

Window-level aggregation

**⚠ Challenge 4: Day-night bias**

Oxygen naturally oscillates

Static thresholds fail

✅ Solution:

Global mean comparison

Window-based voting

CNN learns temporal context

# **✅ Key Takeaway**

The model is not optimized for perfect accuracy but for early and reliable detection of unhealthy states under real operational constraints.

# **Trouble shoot**

In [23]:
from src.train import train_model
y_prob = model.predict(X_val).ravel()
y_true = y_val.ravel()


NameError: name 'X_val' is not defined

In [17]:
import numpy as np
print("X NaNs:", np.isnan(X).any())
print("y NaNs:", np.isnan(y).any())
print("X range:", X.min(), X.max())
print("y shape:", y.shape)


NameError: name 'X' is not defined

In [15]:
X, y = create_sliding_windows_multi(df_scaled)

print("X shape:", X.shape)
print("y shape:", y.shape)

print("NaNs in X:", np.isnan(X).any())
print("NaNs in y:", np.isnan(y).any())

print("X range:", X.min(), X.max())
print("Class balance:", np.unique(y, return_counts=True))


NameError: name 'create_sliding_windows_multi' is not defined

In [18]:
from sklearn.preprocessing import RobustScaler
import numpy as np

def scale_windows(X):
    """
    Scale windowed data safely.
    X shape: (samples, timesteps, features)
    """
    scaler = RobustScaler()

    n_samples, n_steps, n_features = X.shape
    X_reshaped = X.reshape(-1, n_features)

    X_scaled = scaler.fit_transform(X_reshaped)
    return X_scaled.reshape(n_samples, n_steps, n_features)


scale_windows(X)


array([[[ 2.99800743e+00,  2.36263736e+01, -4.75000000e+00, ...,
          2.57425743e-01, -9.01639344e-01, -4.67741935e-01],
        [ 7.97203257e+00,  6.13846154e+00, -4.91666667e+00, ...,
          1.50495050e+00,  1.54098361e+00,  1.62903226e+00],
        [ 3.11862634e+01,  8.94505495e-01, -4.83333333e+00, ...,
         -4.90099010e-01, -7.95081967e-01,  1.77419355e-01],
        ...,
        [ 6.45384918e-01,  3.67032967e-01, -1.66666667e+00, ...,
         -3.31683168e-01, -1.06557377e-01, -2.25806452e-01],
        [ 4.10681947e-01,  4.19780220e-01, -7.50000000e-01, ...,
         -4.90099010e-01, -3.85245902e-01, -1.53225806e-01],
        [ 2.41548184e-01,  4.19780220e-01, -7.50000000e-01, ...,
         -7.42574257e-02, -4.83606557e-01,  5.40322581e-01]],

       [[ 7.97203257e+00,  6.13846154e+00, -4.91666667e+00, ...,
          1.50495050e+00,  1.54098361e+00,  1.62903226e+00],
        [ 3.11862634e+01,  8.94505495e-01, -4.83333333e+00, ...,
         -4.90099010e-01, -7.95081967e

In [19]:
import numpy as np

print("y dtype:", y.dtype)
print("y shape:", y.shape)
print("Unique y:", np.unique(y))
print("NaNs in y:", np.isnan(y).any())

print("Count class 0:", np.sum(y == 0))
print("Count class 1:", np.sum(y == 1))


y dtype: int64
y shape: (50,)
Unique y: [0 1]
NaNs in y: False
Count class 0: 16
Count class 1: 34


In [18]:
df.columns

Index(['irradiance', 'temperature', 'pH', 'oxygen', 'DIC', 'DICVsChloride',
       'nitrate', 'NitrateVsChloride', 'phosphate', 'PhosphateVsChloride',
       'chlorophyll_a', 'density', 'salinity', 'PAM0', 'PAM8', 'PAM16'],
      dtype='object')

Evaluation

In [ ]:
check to understand current folder in memory
# import inspect
# inspect.getsource(create_sliding_windows)



In [21]:
print(y)

[0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
